<a href="https://colab.research.google.com/github/cow-coding/ML-DL-Study/blob/master/Python%EC%9D%84%20%EC%9D%B4%EC%9A%A9%ED%95%9C%20%EA%B0%9C%EC%9D%B8%ED%99%94%20%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C/RS_Chapter2_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# Data load

In [ ]:
DATA = '/content/drive/MyDrive/dataset/rec/'
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(DATA+'u.user', sep='|', names=u_cols, encoding='latin-1')
users = users.set_index('user_id')

In [ ]:
users.head()

,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [ ]:
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\s', 'Comedy', 'Crime', 'Documentary',
          'Drama', 'Fantasy', 'File-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv(DATA+'u.item', sep='|', names=i_cols, encoding='latin-1')
movies = movies.set_index('movie_id')

In [ ]:
movies.head()

,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children\s,Comedy,Crime,Documentary,Drama,Fantasy,File-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [ ]:
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(DATA+'u.data', sep='\t', names=r_cols, encoding='latin-1')
ratings = ratings.set_index('user_id')

In [ ]:
ratings.head()

,movie_id,rating,timestamp
user_id,,,
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596


# best-seller

In [ ]:
def recom_movie1(n_items):
  movie_sort = movie_mean.sort_values(ascending=False)[:n_items]
  recom_movies = movies.loc[movie_sort.index]
  recommendations = recom_movies['title']
  
  return recommendations

# same code
def recom_movie2(n_items):
  return movies.loc[movie_mean.sort_values(ascending=False)[:n_items].index]['title']

In [ ]:
movie_mean = ratings.groupby(['movie_id'])['rating'].mean()
recom_movie1(5)
recom_movie2(5)

movie_id
1293                                      Star Kid (1997)
1467                 Saint of Fort Washington, The (1993)
1653    Entertaining Angels: The Dorothy Day Story (1996)
814                         Great Day in Harlem, A (1994)
1122                       They Made Me a Criminal (1939)
Name: title, dtype: object

# RMSE

In [ ]:
def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true)-np.array(y_pred))**2))

In [ ]:
rmse = []

for user in set(ratings.index):
  y_true = ratings.loc[user]['rating']
  y_pred = movie_mean[ratings.loc[user]['movie_id']]
  accuracy = RMSE(y_true, y_pred)
  rmse.append(accuracy)

print(np.mean(rmse))

0.996007224010567


# user group recommend

In [ ]:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(DATA+'u.user', sep='|', names=u_cols, encoding='latin-1')
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\s', 'Comedy', 'Crime', 'Documentary',
          'Drama', 'Fantasy', 'File-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv(DATA+'u.item', sep='|', names=i_cols, encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(DATA+'u.data', sep='\t', names=r_cols, encoding='latin-1')

In [ ]:
ratings = ratings.drop('timestamp', axis=1)
movies = movies[['movie_id', 'title']]

In [ ]:
from sklearn.model_selection import train_test_split

x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

In [ ]:
def score(model):
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])
  y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
  y_true = np.array(x_test['rating'])

  return RMSE(y_true, y_pred)

In [ ]:
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

In [ ]:
def best_seller(user_id, movie_id):
  try:
    rating = train_mean[movie_id]
  except:
    rating = 3.0
  return rating

In [ ]:
train_mean = x_train.groupby(['movie_id'])['rating'].mean()
score(best_seller)

1.0213522170912057

## group by sex

In [ ]:
merged_ratings = pd.merge(x_train, users)
users = users.set_index('user_id')

g_mean = merged_ratings[['movie_id', 'sex', 'rating']].groupby(['movie_id', 'sex'])['rating'].mean()

In [ ]:
def cf_gender(user_id, movie_id):
  if movie_id in rating_matrix:
    gender = users.loc[user_id]['sex']

    if gender in g_mean[movie_id]:
      gender_rating = g_mean[movie_id][gender]
    else:
      gender_rating = 3.0
  
  else:
    gender_rating = 3.0
  
  return gender_rating

In [ ]:
score(cf_gender)

1.0292918445370014

# Practice

## 2-1

In [ ]:
occ_mean = merged_ratings[['movie_id', 'occupation', 'rating']].groupby(['movie_id', 'occupation'])['rating'].mean()

In [ ]:
def cf_occ(user_id, movie_id):
  if movie_id in rating_matrix:
    occ = users.loc[user_id]['occupation']

    if occ in occ_mean[movie_id]:
      occ_rating = occ_mean[movie_id][occ]
    else:
      occ_rating = 3.0
  
  else:
    occ_rating = 3.0

  return occ_rating

In [ ]:
score(cf_occ)

1.1172206755641187

## 2-2

In [ ]:
occ_g_mean = merged_ratings[['movie_id', 'occupation', 'sex', 'rating']].groupby(['movie_id', 'sex', 'occupation'])['rating'].mean()

In [ ]:
def cf_g_occ(user_id, movie_id):
  if movie_id in rating_matrix:
    occ_g = users.loc[user_id][['occupation', 'sex']]

    
    if occ_g['sex'] in occ_g_mean[movie_id]:
      if occ_g['occupation'] in occ_g_mean[movie_id][occ_g['sex']]:
        occ_g_rating = occ_g_mean[movie_id][occ_g['sex']][occ_g['occupation']]
      else:
        occ_g_rating = 3.0
    else:
      occ_g_rating = 3.0

  else:
    occ_g_rating = 3.0

  return occ_g_rating

In [ ]:
score(cf_g_occ)

1.1393312194745098